In [32]:
#Import modules
import pandas as pd
import requests
from collections import defaultdict

In [33]:
#Extract updated list of SP500 tickers
url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

user_agents = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

response = requests.get(url_sp500, headers=user_agents, timeout=30)
response.raise_for_status()

tables = pd.read_html(response.text)
df_sp500 = tables[0]

sector_to_tickers = defaultdict(list)

for symbol, sector in zip(df_sp500["Symbol"], df_sp500["GICS Sector"]):
    sector_to_tickers[sector].append(symbol)

dict_sp500 = dict(sector_to_tickers)

#Copy dict_sp500 to csv
rows = [
    {"Sector": sector, "Ticker Symbol": ticker}
    for sector, tickers in dict_sp500.items()
    for ticker in tickers
]

df_sp500_output = [{'Sector': 'Industrials', 'Ticker Symbol': 'MMM'},
 {'Sector': 'Industrials', 'Ticker Symbol': 'AOS'},
 {'Sector': 'Industrials', 'Ticker Symbol': 'ALLE'}]

df_sp500_output = pd.DataFrame(df_sp500_output)

df_sp500_output.to_csv("sp500_tickers.csv", index=False)

C:\Users\billg\AppData\Local\Temp\ipykernel_20684\2554492108.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [1]:
df_sp500_output
print('test')

NameError: name 'df_sp500_output' is not defined